### Sequence-to-Point architecture

In [1]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


Importing libraries

In [4]:
from collections import OrderedDict  # a dictionary subclass that remembers the order that keys were first inserted
import numpy as np  
import pandas as pd  
from nilmtk.disaggregate import Disaggregator  # a library for non-intrusive load monitoring
from tensorflow.keras.callbacks import ModelCheckpoint  # a library for saving the model weights during training
from tensorflow.keras.layers import Conv1D, Dense, Dropout, Reshape, Flatten  # layers for building neural network models
from tensorflow.keras.models import Sequential  # a class for building neural network models


seq2point

In [13]:
# define custom exception classes
class SequenceLengthError(Exception):
    pass

class ApplianceNotFoundError(Exception):
    pass

# define the Seq2Point class which is a subclass of Disaggregator
class Seq2Point(Disaggregator):

    def __init__(self, params):
        """
        Initialize the Seq2Point model with the specified parameters.
        """
    #initialize the s2p model with the specified parameters - these are default paremeters.

        # initialize the model name to "Seq2Point"
        self.MODEL_NAME = "Seq2Point"
        # create an ordered dictionary to store the models
        self.models = OrderedDict()
        # create a file prefix for the model weights
        self.file_prefix = "{}-temp-weights".format(self.MODEL_NAME.lower())
        # set the chunk-wise training parameter to False by default
        self.chunk_wise_training = params.get('chunk_wise_training',False)
        # set the sequence length parameter to 99 by default
        self.sequence_length = params.get('sequence_length',99)
        # set the number of epochs parameter to 10 by default
        self.n_epochs = params.get('n_epochs', 10 )
        # set the batch size parameter to 512 by default
        self.batch_size = params.get('batch_size',512)
        # set the appliance parameters to an empty dictionary by default
        self.appliance_params = params.get('appliance_params',{})
        # set the mains mean parameter to 1800 by default
        self.mains_mean = params.get('mains_mean',1800)
        # set the mains standard deviation parameter to 600 by default
        self.mains_std = params.get('mains_std',600)
        # Check if the sequence length is odd
        if self.sequence_length%2==0: # Ensuring that sequence length is odd
            # If it's even, raise an exception
            print ("Sequence length should be odd!")
            raise (SequenceLengthError)

#train the s2p model using the given training data

    def partial_fit(self, train_main, train_appliances, do_preprocessing=True, current_epoch=0, **load_kwargs):
        """
        Train the Seq2Point model using the given training data.
        """
        # If no appliance wise parameters are provided, then copmute them using the first chunk
        if len(self.appliance_params) == 0:
            self.set_appliance_params(train_appliances)

        print("...............Seq2Point partial_fit running...............")
        
        # Do the pre-processing, such as  windowing and normalizing
        if do_preprocessing:
            train_main, train_appliances = self.call_preprocessing(
                train_main, train_appliances, 'train')
       
    # Reshape the training data for input to the neural network
        train_main = pd.concat(train_main, axis=0)
        train_main = train_main.values.reshape((-1, self.sequence_length, 1))
        new_train_appliances = []
        for app_name, app_df in train_appliances:
            app_df = pd.concat(app_df, axis=0)
            app_df_values = app_df.values.reshape((-1, 1))
            new_train_appliances.append((app_name, app_df_values))
        train_appliances = new_train_appliances

        for appliance_name, power in train_appliances:
            # Check if the appliance was already trained. If not then create a new model for it
            if appliance_name not in self.models:
                print("First model training for", appliance_name)
                self.models[appliance_name] = self.return_network()
            # Retrain the particular appliance
            else:
                print("Started Retraining model for", appliance_name)

            model = self.models[appliance_name]
            if train_main.size > 0:
                # Sometimes chunks can be empty after dropping NANS
                if len(train_main) > 10:
                    # Do validation when you have sufficient samples
                    filepath = self.file_prefix + "-{}-epoch{}.h5".format(
                            "_".join(appliance_name.split()),
                            current_epoch,
                    )
                    checkpoint = ModelCheckpoint(filepath,monitor='val_loss',verbose=1,save_best_only=True,mode='min')
                    model.fit(
                            train_main, power,
                            validation_split=0.15,
                            epochs=self.n_epochs,
                            batch_size=self.batch_size,
                            callbacks=[checkpoint],
                    )
                    model.load_weights(filepath)

                    
    def disaggregate_chunk(self,test_main_list,model=None,do_preprocessing=True):
        """
        Test the model with given test data
        """
        #Explanation:
# This method is used to disaggregate a chunk of test data using the trained Seq2Point model. If a model is already
# provided, it uses that model. Otherwise, it uses the model stored in the Seq2Point object. Before disaggregating,
# the method preprocesses the test mains such as windowing and normalizing. Then, it predicts the power consumption
# of each appliance using the model and stores the results in a dictionary. Finally, the dictionary is converted to a
# pandas DataFrame and the results are returned.

        if model is not None:
            self.models = model

        # Preprocess the test mains such as windowing and normalizing

        if do_preprocessing:
            test_main_list = self.call_preprocessing(test_main_list, submeters_lst=None, method='test')

        test_predictions = []
        for test_main in test_main_list:
            test_main = test_main.values
            test_main = test_main.reshape((-1, self.sequence_length, 1))
            disggregation_dict = {}
            for appliance in self.models:
                prediction = self.models[appliance].predict(test_main,batch_size=self.batch_size)
                prediction = self.appliance_params[appliance]['mean'] + prediction * self.appliance_params[appliance]['std']
                valid_predictions = prediction.flatten()
                valid_predictions = np.where(valid_predictions > 0, valid_predictions, 0)
                df = pd.Series(valid_predictions)
                disggregation_dict[appliance] = df
            results = pd.DataFrame(disggregation_dict, dtype='float32')
            test_predictions.append(results)
        return test_predictions

    def return_network(self):
        """
        Define the neural network model
        """
        
        # Model architecture
        
        #create a sequential model
        model = Sequential() 
        # add a 1D convolutional layer with 30 filters, kernel size of 10, ReLU activation, and input shape of (self.sequence_length, 1)
        model.add(Conv1D(30,10,activation="relu",input_shape=(self.sequence_length,1),strides=1))
        # add another 1D convolutional layer with 30 filters, kernel size of 8, and ReLU activation
        model.add(Conv1D(30, 8, activation='relu', strides=1))
        # add another 1D convolutional layer with 40 filters, kernel size of 6, and ReLU activation
        model.add(Conv1D(40, 6, activation='relu', strides=1))
         # add another 1D convolutional layer with 50 filters, kernel size of 5, and ReLU activation
        model.add(Conv1D(50, 5, activation='relu', strides=1))
        # add a dropout layer with a rate of 0.2
        model.add(Dropout(.2))
        # add another 1D convolutional layer with 50 filters, kernel size of 5, and ReLU activation
        model.add(Conv1D(50, 5, activation='relu', strides=1))
        # add another dropout layer with a rate of 0.2
        model.add(Dropout(.2))
        # flatten the output of the previous layers
        model.add(Flatten())
        # add a dense layer with 1024 neurons and ReLU activation
        model.add(Dense(1024, activation='relu'))
        # add another dropout layer with a rate of 0.2
        model.add(Dropout(.2))
         # add a dense layer with 1 neuron (for output) and no activation function
        model.add(Dense(1))
        # compile the model with mean squared error loss and Adam optimizer
        model.compile(loss='mse', optimizer='adam')  # ,metrics=[self.mse])
        # return the compiled model
        return model

    def call_preprocessing(self, mains_lst, submeters_lst, method):
        """
        Preprocess the given data
        """
        if method == 'train':
            # Preprocessing for the train data
            
            mains_df_list = []
            # loop over the mains signals and preprocess them
            for mains in mains_lst:
                # Flattening the 2D dataframe into a 1D array
                new_mains = mains.values.flatten()
                n = self.sequence_length
                units_to_pad = n // 2
                # Padding zeros to ensure the first and last data points are used as a part of the window
                new_mains = np.pad(new_mains,(units_to_pad,units_to_pad),'constant',constant_values=(0,0))
                # Splitting the data into overlapping windows of length sequence_length
                new_mains = np.array([new_mains[i:i + n] for i in range(len(new_mains) - n + 1)])
                # Normalize the data
                new_mains = (new_mains - self.mains_mean) / self.mains_std
                mains_df_list.append(pd.DataFrame(new_mains))

            appliance_list = []
            for app_index, (app_name, app_df_list) in enumerate(submeters_lst):
                if app_name in self.appliance_params:
                    app_mean = self.appliance_params[app_name]['mean']
                    app_std = self.appliance_params[app_name]['std']
                else:
                    # Raise an exception if the appliance is not found
                    print ("Parameters for ", app_name ," were not found!")
                    raise ApplianceNotFoundError()

                processed_appliance_dfs = []

                for app_df in app_df_list:
                    # Normalize the data
                    new_app_readings = app_df.values.reshape((-1, 1))
                    # This is for choosing windows
                    new_app_readings = (new_app_readings - app_mean) / app_std  
                    # Return as a list of dataframe
                    processed_appliance_dfs.append(pd.DataFrame(new_app_readings))
                appliance_list.append((app_name, processed_appliance_dfs))
            return mains_df_list, appliance_list

        else:
            # Preprocessing for the test data
            mains_df_list = []

            for mains in mains_lst:
                # Flattening the 2D dataframe into a 1D array
                new_mains = mains.values.flatten()
                n = self.sequence_length
                units_to_pad = n // 2
                # Padding zeros to ensure the first and last data points are used as a part of the window
                new_mains = np.pad(new_mains,(units_to_pad,units_to_pad),'constant',constant_values=(0,0))
                # Splitting the data into overlapping windows of length sequence_length
                new_mains = np.array([new_mains[i:i + n] for i in range(len(new_mains) - n + 1)])
                # Normalize the data
                new_mains = (new_mains - self.mains_mean) / self.mains_std
                mains_df_list.append(pd.DataFrame(new_mains))
            return mains_df_list

    def set_appliance_params(self,train_appliances):
        """
        Set the appliance parameters such as mean and std deviation
        """
        # Loop through each appliance and its dataframes in the training data
        for (app_name,df_list) in train_appliances:
            # Concatenate all dataframes into one numpy array
            l = np.array(pd.concat(df_list,axis=0))
             # Calculate the mean and standard deviation of the concatenated array
            app_mean = np.mean(l)
            app_std = np.std(l)
            #If the standard deviation is too small, set it to 100 (arbitrary value)
            if app_std<1:
                app_std = 100
            # Add the mean and std deviation to the appliance parameters dictionary
            self.appliance_params.update({app_name:{'mean':app_mean,'std':app_std}})
        # Print the appliance parameters dictionary
        print (self.appliance_params)

### Seq2Point model:

1. An instance of the Seq2Point class is created with the specified parameters.
2. The partial_fit method is called with the training data to train the Seq2Point model.
3. If no appliance wise parameters are provided, the set_appliance_params method is called to compute them using the first chunk of the training data.
4. The call_preprocessing method is called to preprocess the training data.
5. The training data is reshaped for input to the neural network and new train appliances are created with preprocessed values.
6. The return_network method is called to define the neural network model.
7. The neural network model is compiled with mean squared error loss and Adam optimizer.
8. The fit method is called on the model to train it on the training data for each appliance.
9. The trained model weights are saved in the form of a file.
10. The disaggregate_chunk method is called with the test data to disaggregate it using the trained Seq2Point model.
11. The call_preprocessing method is called to preprocess the test data.
12. The test data is disaggregated using the trained model and the results are stored in a dictionary.
13. The dictionary is converted to a pandas DataFrame and the results are returned.

Some steps may be repeated depending on the number of chunks of training and test data provided.

### Train & Test split

The code provided for Seq2Point does not include a function to split the data into training and testing sets. Instead, it assumes that you have already split your data into training and testing sets and provided those sets as inputs to the partial_fit() and disaggregate_chunk() methods.

### Model architecture

This architecture is a deep neural network designed for time series data. It uses a series of one-dimensional convolutional layers to extract relevant features from the input data, each with different filter sizes and numbers of filters. The filters are essentially small weight matrices that slide over the input data, looking for patterns that are useful for the task at hand.

The kernel size parameter determines the width of the filter (i.e., how many time steps to look at together), while the number of filters determines how many different patterns the layer can learn. ReLU activation function is used in each convolutional layer, which is a simple non-linear function that introduces non-linearity into the model.

Dropout is a regularization technique that helps prevent overfitting by randomly dropping out some of the neurons during training. This is done to reduce the co-adaptation of neurons and force the network to learn more robust features.

The Flatten layer is used to convert the output of the convolutional layers into a 1-dimensional array that can be passed through a fully connected layer. A fully connected layer (Dense) with 1024 neurons and ReLU activation is then added, followed by another dropout layer to further prevent overfitting.

Finally, a dense layer with just one neuron is added, which produces the output of the model. This neuron is not activated (i.e., no activation function is applied), because we want the output to be a continuous value.